In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
!python -m spacy download en_core_web_lg

2024-02-01 04:34:07.454539: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-01 04:34:07.454598: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-01 04:34:07.457134: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-01 04:34:07.468256: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-02-01 04:34:10.026489: W tensorflow/compiler/tf2

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install --upgrade pandas==1.5.1
!pip install -q pyLDAvis==3.4.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 46.0 MB/s eta 0:00:00
  Attempting uninstall: pandas
    Found existing installation: pandas 1.5.3
    Uninstalling pandas-1.5.3:
      Successfully uninstalled pandas-1.5.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lida 0.0.10 requires fastapi, which is not installed.
lida 0.0.10 requires kaleido, which is not installed.
lida 0.0.10 requires python-multipart, which is not installed.
lida 0.0.10 requires uvicorn, which is not installed.
google-colab 1.0.0 requires pandas==1.5.3, but you have pandas 1.5.1 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 10.3 MB/s eta 0:00:00


In [ ]:
# Code cell 1
import spacy
import nltk
import os
import pandas as pd
import numpy as np
from nltk.tokenize import RegexpTokenizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
import seaborn as sb
import pyLDAvis.lda_model
from gensim import corpora, models
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# Load the CSV file into a DataFrame
rra = r'/content/drive/MyDrive/restaurant_reviews_az.csv'
df= pd.read_csv(rra)

# Summary
print(df.head(5))
print(df.shape)
print(df.describe())

                review_id                 user_id             business_id  \
0  IVS7do_HBzroiCiymNdxDg  fdFgZQQYQJeEAshH4lxSfQ  sGy67CpJctjeCWClWqonjA   
1  QP2pSzSqpJTMWOCuUuyXkQ  JBLWSXBTKFvJYYiM-FnCOQ  3w7NRntdQ9h0KwDsksIt5Q   
2  oK0cGYStgDOusZKz9B1qug  2_9fKnXChUjC5xArfF8BLg  OMnPtRGmbY8qH_wIILfYKA   
3  E_ABvFCNVLbfOgRg3Pv1KQ  9MExTQ76GSKhxSWnTS901g  V9XlikTxq0My4gE8LULsjw   
4  Rd222CrrnXkXukR2iWj69g  LPxuausjvDN88uPr-Q4cQA  CA5BOxKRDPGJgdUQ8OUOpw   

   stars  useful  funny  cool  \
0      3       1      1     0   
1      5       1      1     1   
2      5       1      0     0   
3      5       0      0     0   
4      4       1      0     0   

                                                text                 date  
0  OK, the hype about having Hatch chili in your ...  2020-01-27 22:59:06  
1  Pandemic pit stop to have an ice cream.... onl...  2020-04-19 05:33:16  
2  I was lucky enough to go to the soft opening a...  2020-02-29 19:43:44  
3  I've gone to claim Jumpers all 

In [ ]:
# Code cell 2
selected_reviews = df[(df['stars'] == 1) | (df['stars'] == 5)]

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
# Code cell 3
# Perform text preprocessing - remove stopwords and tokenize
nltk.download('stopwords')
nltk.download('punkt')
def preprocess_text(text):
    stop_words = set(stopwords.words('english'))
    # Tokenize and remove stopwords and punctuation
    tokens = word_tokenize(text.lower())
    tokens = [word for word in tokens if word.isalpha() and word not in stop_words]
    return ' '.join(tokens)

selected_reviews['processed_text'] = selected_reviews['text'].apply(preprocess_text)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
#Code cell 4
# Use Count Vectorizer to represent the reviews
vectorizer = CountVectorizer(max_features=5000)
X = vectorizer.fit_transform(selected_reviews['processed_text'])

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
# Code cell 5
feature_names = vectorizer.get_feature_names_out()
word_counts = X.toarray()

one_star_counts = word_counts[selected_reviews['stars'] == 1].sum(axis=0)
five_star_counts = word_counts[selected_reviews['stars'] == 5].sum(axis=0)

one_star_top_words = [feature_names[idx] for idx in one_star_counts.argsort()[::-1][:20]]
five_star_top_words = [feature_names[idx] for idx in five_star_counts.argsort()[::-1][:20]]

print("Top 20 words in 1-star reviews:", one_star_top_words)
print("Top 20 words in 5-star reviews:", five_star_top_words)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Top 20 words in 1-star reviews: ['food', 'order', 'time', 'get', 'would', 'place', 'back', 'service', 'us', 'one', 'like', 'ordered', 'go', 'never', 'even', 'got', 'said', 'minutes', 'could', 'restaurant']
Top 20 words in 5-star reviews: ['food', 'great', 'place', 'good', 'service', 'delicious', 'amazing', 'back', 'best', 'time', 'tucson', 'love', 'also', 'go', 'always', 'one', 'friendly', 'staff', 'restaurant', 'definitely']


In [ ]:
# Code Cells 6
# Set the number of topics to 10 and train an LDA model for 10 iterations
num_topics = 10
lda = LatentDirichletAllocation(n_components=num_topics, random_state=42, n_jobs=-1, max_iter=10)
lda.fit(X)


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


LatentDirichletAllocation(n_jobs=-1, random_state=42)

In [ ]:
# Code Cell 7
# Show top 10 words generated by each topic
for i, topic in enumerate(lda.components_):
    top_words_idx = topic.argsort()[::-1][:10]
    top_words = [feature_names[idx] for idx in top_words_idx]
    print(f"Topic {i+1}: {', '.join(top_words)}")

Topic 1: us, food, minutes, back, came, service, asked, table, said, would
Topic 2: order, food, time, get, would, service, customer, go, location, called
Topic 3: great, dinner, us, restaurant, patio, delicious, night, menu, nice, wine
Topic 4: chicken, good, delicious, food, rice, also, ordered, sauce, spicy, fresh
Topic 5: pizza, food, like, chicken, place, ordered, wings, good, sauce, got
Topic 6: food, great, service, place, amazing, friendly, good, delicious, staff, back
Topic 7: burger, fries, sandwich, cheese, salad, ordered, bacon, steak, bread, got
Topic 8: tacos, food, good, mexican, place, salsa, burrito, taco, best, delicious
Topic 9: always, place, food, tucson, love, best, one, go, get, good
Topic 10: coffee, cream, like, ice, tea, breakfast, good, place, got, also


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
# Code Cell 8
# Show the document topic distribution of the first 5 reviews in your dataset
document_topic_distribution = lda.transform(X[:5])

for i, review in enumerate(selected_reviews['processed_text'][:5]):
    print(f"Review {i+1}: {review}")
    print("Topic Distribution:", document_topic_distribution[i])
    print("="*50)

Review 1: pandemic pit stop ice cream plain sundae limited menu written screens outside unpleasant surprise cashier wearing gloves mask holding item good since hold lid three customers pm location bomb parking access easy great visibility pictures tonight soo
Topic Distribution: [0.00277857 0.24790169 0.00277878 0.00277831 0.00277837 0.00277872
 0.00277823 0.00277859 0.34587146 0.38677728]
Review 2: lucky enough go soft opening let tell good beer wine many different modern italian appetizers bruschettas paninis salads please favor visit place good staff friendly look forward enjoying restaurant future
Topic Distribution: [0.00344896 0.00344897 0.56710854 0.00344908 0.00344918 0.0034494
 0.0034489  0.00344911 0.40529867 0.00344919]
Review 3: gone claim jumpers us never disappoint location different cook food well excellent desserts service deliver everything impeccable would absolutely recommend restaurant quality meal best place tucson take wife dinner hands
Topic Distribution: [0.0033

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
# Code Cell 9: Experiment with different number of topics and iterations
best_model = None
best_score = -1  # Initialize with a low value

# Adjust the range and step size for quicker experimentation
for num_topics in range(3, 6):  # Experiment with 3 to 5 topics
    for num_iterations in range(5, 11, 2):  # Experiment with iterations from 5 to 10 in steps of 2
        vectorizer = CountVectorizer(max_features=1000)  # Adjust max_features based on available memory
        X = vectorizer.fit_transform(selected_reviews['processed_text'])

        lda = LatentDirichletAllocation(n_components=num_topics, random_state=42, n_jobs=-1, max_iter=num_iterations)
        lda.fit(X)

        # Use perplexity as a scoring metric
        perplexity_score = lda.perplexity(X)

        print(f"Number of Topics: {num_topics}, Iterations: {num_iterations}, Perplexity: {perplexity_score}")

        if perplexity_score < best_score or best_score == -1:
            best_score = perplexity_score
            best_model = lda

print(f"\nBest Model Configuration: Number of Topics = {best_model.n_components}, Iterations = {best_model.max_iter}")

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Number of Topics: 3, Iterations: 5, Perplexity: 536.456299893303
Number of Topics: 3, Iterations: 7, Perplexity: 532.1865636338417
Number of Topics: 3, Iterations: 9, Perplexity: 530.5813157114258
Number of Topics: 4, Iterations: 5, Perplexity: 545.0654018446597
Number of Topics: 4, Iterations: 7, Perplexity: 538.5424189633152
Number of Topics: 4, Iterations: 9, Perplexity: 535.2640860150893
Number of Topics: 5, Iterations: 5, Perplexity: 549.8309606374969
Number of Topics: 5, Iterations: 7, Perplexity: 542.6128898252567
Number of Topics: 5, Iterations: 9, Perplexity: 538.6311079661227

Best Model Configuration: Number of Topics = 3, Iterations = 9


In [ ]:
panel = pyLDAvis.lda_model.prepare(best_model, X, vectorizer, mds='tsne')
pyLDAvis.display(panel)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
# Code Cell 10: Show the top 10 words for each topic and document topic distribution of the first 5 reviews based on the best model
for i, topic in enumerate(best_model.components_):
    top_words_idx = topic.argsort()[::-1][:10]
    top_words = [feature_names[idx] for idx in top_words_idx]
    print(f"Topic {i+1}: {', '.join(top_words)}")

document_topic_distribution_best = best_model.transform(X[:5])

for i, review in enumerate(selected_reviews['processed_text'][:5]):
    print(f"Review {i+1}: {review}")
    print("Topic Distribution:", document_topic_distribution_best[i])
    print("="*50)

Topic 1: busier, banana, compare, cod, basil, contained, burgers, admit, cheers, brussel
Topic 2: bean, ample, business, areas, beans, accommodate, changes, amount, banana, canelo
Topic 3: banana, beats, canelo, cheers, bean, accompanied, accommodating, commented, african, baristas
Review 1: pandemic pit stop ice cream plain sundae limited menu written screens outside unpleasant surprise cashier wearing gloves mask holding item good since hold lid three customers pm location bomb parking access easy great visibility pictures tonight soo
Topic Distribution: [0.57368238 0.23334336 0.19297426]
Review 2: lucky enough go soft opening let tell good beer wine many different modern italian appetizers bruschettas paninis salads please favor visit place good staff friendly look forward enjoying restaurant future
Topic Distribution: [0.01602784 0.14795039 0.83602177]
Review 3: gone claim jumpers us never disappoint location different cook food well excellent desserts service deliver everything im

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)



The first topic appears to cover aspects like crowd levels, food ingredients, and the overall restaurant atmosphere. It might capture reviews discussing the busy environment, specific dishes, and the overall vibe. The second topic revolves around business-related elements, including the overall environment, changes, and the variety of items available such as beans and banana. It likely touches on aspects related to the restaurant's offerings and flexibility. The third topic is a bit more complex, suggesting an emphasis on entertainment or unique experiences within the restaurant. Words like "beats," "cheers," and "accompanying" hint at a lively atmosphere, possibly with cultural influences. The reviews vary in sentiment, with one expressing negative sentiments linked to the pandemic, another providing a positive outlook on a soft opening, and others praising specific restaurants for their quality service and diverse offerings. Yet, a negative experience during July at a hotel casino stands out, contrasting with a positive surprise during a town visit, highlighting well-seasoned burgers and friendly service.


In [ ]:
!pip install -q jupyter
!pip install -q nbconvert

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.4/123.4 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.5/93.5 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 11.3 MB/s eta 0:00:00
